In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [8]:
df=pd.read_csv("ECB_FX_USD-quote.csv")
df=df.dropna(axis=1).drop(0)
for column in df.columns:
    if column =="Date":
        df[column] = pd.to_datetime(df[column])
    else:
        df[column]=df[column].apply(float)
df=df.sort_values("Date").reset_index(drop=True)


Null model

In [10]:
null = pd.DataFrame({"EUR_dbefore":df["EUR"].shift(),"EUR_day":df["EUR"]}).dropna()

In [6]:
null.iloc[0,0]

'1.1789'

In [12]:
null["Squared_error"] = null.apply(lambda x: (x["EUR_day"]-x["EUR_dbefore"])**2,axis=1)
(null["Squared_error"].sum()/len(null))**0.5

0.007174079683256437

Neural networks

In [13]:
len(df)

6073

In [17]:
df.iloc[:30]

,Date,EUR,JPY,CZK,DKK,GBP,HUF,PLN,SEK,CHF,NOK,AUD,CAD,HKD,KRW,NZD,SGD,ZAR
0,1999-01-04,1.1789,0.008815524,0.033580198,0.158239487,1.657854029,0.004687848,0.289570643,0.124493115,0.729156358,0.133133823,0.617225131,0.654798934,0.129078527,0.00084292,0.530343245,0.602894548,0.169973183
1,1999-01-05,1.179,0.009002749,0.033765787,0.158265655,1.655433867,0.004700957,0.292955647,0.125392183,0.731253489,0.134366631,0.622360642,0.656276092,0.129076756,0.000858697,0.53564127,0.599847367,0.17344612
2,1999-01-06,1.1743,0.008935474,0.033695839,0.157725783,1.65955342,0.004684645,0.293098715,0.126200967,0.728654753,0.134459266,0.623963868,0.663034272,0.129029777,0.000863748,0.536455002,0.596121631,0.174469223
3,1999-01-07,1.1632,0.008987097,0.033342888,0.15627897,1.647942197,0.004651126,0.289605378,0.12671024,0.719579338,0.134793441,0.629641659,0.660833996,0.129056595,0.000869903,0.540244299,0.598477053,0.170349867
4,1999-01-08,1.1659,0.008962257,0.033370542,0.156637513,1.643501551,0.004660804,0.288853653,0.12721222,0.722456314,0.13572759,0.63343475,0.660828657,0.129111205,0.000853058,0.540845201,0.596765112,0.171822268
5,1999-01-11,1.1569,0.009157761,0.032891707,0.155428372,1.642390687,0.00463316,0.286929563,0.127152827,0.718392946,0.135175556,0.637972869,0.6624864,0.129039094,0.000852731,0.544244249,0.597634053,0.170357827
6,1999-01-12,1.152,0.008886832,0.032680851,0.1547763,1.630342485,0.004622793,0.285643442,0.126656039,0.716774515,0.133797909,0.63681592,0.662373505,0.129081415,0.000851171,0.543140028,0.595564287,0.165562438
7,1999-01-13,1.1744,0.00890439,0.032520145,0.157781599,1.65782044,0.00460531,0.279399519,0.128490153,0.737179085,0.134263176,0.630719656,0.648016333,0.129043601,0.000852293,0.540103017,0.592682311,0.161890189
8,1999-01-14,1.1653,0.008778816,0.032643285,0.156548491,1.650099122,0.004590687,0.281916052,0.127564313,0.731329233,0.133566393,0.634937068,0.658101316,0.129064771,0.000843058,0.540241076,0.594086159,0.163399518
9,1999-01-15,1.1626,0.008824958,0.032420524,0.156177375,1.651655065,0.00458909,0.276809524,0.126962979,0.727352352,0.13340218,0.63226017,0.653256167,0.129031542,0.00084506,0.538689649,0.59407256,0.161031622
